# THIS FILE IS IN THE HANDOUTS FOLDER. COPY IT INTO YOUR CLASS NOTES

- [**Read the chapter on the website!**](https://ledatascifi.github.io/ledatascifi-2023/content/05/02_reg.html) It contains a lot of extra information we won't cover in class extensively.
- After reading that, I recommend [this webpage as a complimentary place to get additional intuition.](https://aeturrell.github.io/coding-for-economists/econmt-regression.html)

## ASAP

[Declare your team and project interests in the project sheet](https://docs.google.com/spreadsheets/d/1kRbuRKfKh9lCdoVBGLxSbDTIRBEfnV7Y8AcP-hZbmTw/edit?usp=sharing)


# Today: Regression

We start our machine learning applications with regression for a few simple reasons:
- Regression is fundamental method for estimating the relationship between a variable ("y") that condition on many ("X") variables. 
- But the coefficients obtained can also be used to generate predictions. 
- _Note: The focus in this section is on RELATIONSHIP paradigm_
- Many issues that confront researchers have well understood solutions when regression is the model being used. 
- Regression coefficients are easy to interpret.


  
## Objectives

1. You can fit a regression with `statsmodels` or `sklearn`
    - statsmodels: Nicer result tables, usually easier to specifying the regression model
    - sklearn: Easier to use within a prediction/ML exercise
2. You can view the results visually or numerically of your model with either method
3. The focus today is on the _mechanics_ of running regressions, viewing the output, and using the estimation's output objects.

![](https://media.giphy.com/media/yoJC2K6rCzwNY2EngA/giphy.gif)


In [5]:
import numpy as np
import pandas as pd
import seaborn as sns
import statsmodels.api as sm
from sklearn.linear_model import LinearRegression
from statsmodels.formula.api import ols as sm_ols
import matplotlib.pyplot as plt


In [4]:
!pip install statsmodels 

     ---------------------------------------- 9.1/9.1 MB 23.4 MB/s eta 0:00:00
     ------------------------------------- 233.8/233.8 kB 14.9 MB/s eta 0:00:00


## Data

First, we load the data. 

**This is a new dataset, so we should do some data exploration!** Things students should try:
- describe() - any impossible values
- value_count() any categorical variables
- didn't we have a community function to start the EDA?
- correlation heat map
- look for outliers for all variables, and within pairplots
- print out and explore many sections of the data manually (in Excel or Spyder) to get familiar and check for data consistency issues


In [6]:
url = 'https://github.com/LeDataSciFi/data/raw/main/Fannie%20Mae/Fannie_Mae_Plus_Data.gzip?raw=true'
fannie_mae = pd.read_csv(url,compression='gzip') 

## Task 1

Spend 5 minutes exploring the data and jot down what you learn about the data. 


In [18]:
fannie_mae.describe()


,Loan_Identifier,Original_Interest_Rate,Original_UPB,Original_Loan_Term,Original_LTV_(OLTV),Original_Combined_LTV_(CLTV),Number_of_Borrowers,Original_Debt_to_Income_Ratio,Borrower_Credit_Score_at_Origination,Number_of_units,...,CPIAUCSL,rGDP,TCMR,POILWTIUSDM,TTLCONS,DEXUSEU,BOPGSTB,GOLDAMGBD228NLBM,CSUSHPISA,MSPUS
count,1.350380e+05,135038.000000,1.350380e+05,135038.000000,135038.000000,134007.000000,135007.00000,132396.000000,134481.000000,135038.000000,...,135038.000000,135038.000000,135038.000000,135038.000000,1.350380e+05,135038.000000,135038.000000,135038.000000,135038.000000,135038.000000
mean,5.518023e+11,5.238376,1.889311e+05,307.064826,70.057295,70.860858,1.58791,33.298733,742.428797,1.035027,...,208.188326,2.083868,3.478040,56.277796,9.631193e+05,1.180771,-42336.575453,845.949803,148.634283,231137.664954
std,2.597821e+11,1.289895,1.087424e+05,82.331674,17.493178,17.566607,0.50841,11.508698,53.428076,0.244345,...,24.776994,2.310579,1.207100,27.841047,1.538464e+05,0.172754,9404.518716,498.115360,24.712708,46186.084143
min,1.000020e+11,2.250000,8.000000e+03,60.000000,4.000000,4.000000,1.00000,1.000000,361.000000,1.000000,...,164.700000,-8.400000,1.504000,11.990000,7.088180e+05,0.852538,-67823.000000,256.197727,93.236000,157400.000000
25%,3.270665e+11,4.250000,1.080000e+05,240.000000,60.000000,61.000000,1.00000,25.000000,707.000000,1.000000,...,183.100000,0.900000,2.324545,30.710000,8.467770e+05,1.072658,-45943.000000,350.765217,130.151000,190100.000000
50%,5.525325e+11,5.250000,1.640000e+05,360.000000,75.000000,75.000000,2.00000,33.000000,755.000000,1.000000,...,212.495000,2.200000,3.675000,48.748636,8.912640e+05,1.191335,-41360.000000,857.726190,145.632000,224100.000000
75%,7.773282e+11,6.125000,2.470000e+05,360.000000,80.000000,80.000000,2.00000,42.000000,786.000000,1.000000,...,231.797000,3.500000,4.347619,81.899524,1.101187e+06,1.316019,-36519.000000,1273.579545,169.868000,258400.000000
max,9.999850e+11,11.000000,1.170000e+06,360.000000,97.000000,142.000000,8.00000,64.000000,850.000000,4.000000,...,251.176000,7.500000,6.661000,133.927143,1.335425e+06,1.575864,-15946.000000,1780.647727,202.411000,337900.000000


In [22]:
#len(fannie_mae)==fannie_mae["Loan_Identifier"].unique()
fannie_mae.columns
fannie_mae["Origination_Channel"].value_counts()
fannie_mae["Seller_Name"].unique()
fannie_mae["Seller_Name"].value_counts().sort_index()
fannie_mae[["Origination_Date","Qdate"]]
pd.to_datetime(fannie_mae["Origination_Date"]).dt.year.value_counts().sort_index()

1999      537
2000     4367
2001    11341
2002    13069
2003    17363
2004     5936
2005     4928
2006     3663
2007     4275
2008     5086
2009     8014
2010     6606
2011     5634
2012     9102
2013     7533
2014     4875
2015     6318
2016     7755
2017     6505
2018     2131
Name: Origination_Date, dtype: int64

## Clean the data and create variables we will use

These variables are pretty straightforward:

In [24]:
fannie_mae = (fannie_mae
                  # create variables
                  .assign(l_credscore = np.log(fannie_mae['Borrower_Credit_Score_at_Origination']),
                          l_LTV = np.log(fannie_mae['Original_LTV_(OLTV)']),
                          Origination_Date = lambda x: pd.to_datetime(x['Origination_Date']),
                          Origination_Year = lambda x: x['Origination_Date'].dt.year,
                          const = 1,
                          great = fannie_mae['Borrower_Credit_Score_at_Origination'] >= 800
                         )
              
             )
fannie_mae

,Loan_Identifier,Origination_Channel,Seller_Name,Original_Interest_Rate,Original_UPB,Original_Loan_Term,Original_LTV_(OLTV),Original_Combined_LTV_(CLTV),Number_of_Borrowers,Original_Debt_to_Income_Ratio,...,DEXUSEU,BOPGSTB,GOLDAMGBD228NLBM,CSUSHPISA,MSPUS,l_credscore,l_LTV,Origination_Year,const,great
0,9.733730e+11,B,OTHER,6.875,32000.0,360.0,90.0,90.0,1.0,22.0,...,1.308021,-58478.0,665.1025,184.601,257400.0,6.505784,4.499810,2007,1,False
1,9.276200e+11,B,"PNC BANK, N.A.",5.875,200000.0,360.0,80.0,80.0,2.0,26.0,...,1.308021,-58478.0,665.1025,184.601,257400.0,6.541030,4.382027,2007,1,False
2,7.176670e+11,B,OTHER,6.250,122000.0,180.0,80.0,80.0,2.0,31.0,...,1.308021,-58478.0,665.1025,184.601,257400.0,6.608001,4.382027,2007,1,False
3,9.889510e+11,C,AMTRUST BANK,6.000,67000.0,180.0,77.0,77.0,2.0,17.0,...,1.308021,-58478.0,665.1025,184.601,257400.0,6.689599,4.343805,2007,1,True
4,1.908850e+11,R,OTHER,5.875,50000.0,180.0,41.0,41.0,2.0,10.0,...,1.308021,-58478.0,665.1025,184.601,257400.0,6.489205,3.713572,2007,1,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
135033,9.204900e+11,R,OTHER,4.625,200000.0,240.0,50.0,50.0,1.0,45.0,...,1.167900,-47431.0,1299.1500,202.411,315600.0,6.575076,3.912023,2018,1,False
135034,9.666890e+11,R,OTHER,4.625,94000.0,360.0,47.0,47.0,1.0,39.0,...,1.167900,-47431.0,1299.1500,202.411,315600.0,6.517671,3.850148,2018,1,False
135035,6.616280e+11,R,OTHER,4.625,239000.0,360.0,74.0,74.0,2.0,20.0,...,1.167900,-47431.0,1299.1500,202.411,315600.0,6.669498,4.304065,2018,1,False
135036,5.102850e+11,R,OTHER,5.000,93000.0,360.0,44.0,44.0,1.0,19.0,...,1.167900,-47431.0,1299.1500,202.411,315600.0,6.431331,3.784190,2018,1,False


Credit rating is a number between 0 and 850. But in some analysis, it might make sense to have categories of credit ratings (e.g. bad to good). I borrowed [these cutoffs from experian.](https://www.experian.com/blogs/ask-experian/infographic-what-are-the-different-scoring-ranges/)

In [25]:
# create a categorical bin var with "pd.cut()"

fannie_mae['creditbins']= pd.cut(fannie_mae['Borrower_Credit_Score_at_Origination'],
                                 [0,579,669,739,799,850],
                                 labels=['Very Poor','Fair','Good','Very Good','Exceptional'])

Here is the variable that created. I notice that 669 (right on the threshold of a bin) goes into the "Fair" bin instead of "Good".

In [26]:
fannie_mae.loc[:5,['Borrower_Credit_Score_at_Origination','creditbins']]

,Borrower_Credit_Score_at_Origination,creditbins
0,669.0,Fair
1,693.0,Good
2,741.0,Very Good
3,804.0,Exceptional
4,658.0,Fair
5,665.0,Fair


In [27]:
# pd.cut took credit , var number between 0 and 850,
# and changed it to bins. I labeled the bins explicitly

fannie_mae['creditbins'].value_counts(dropna=False)

Very Good      63855
Good           39539
Exceptional    15889
Fair           14560
Very Poor        638
NaN              557
Name: creditbins, dtype: int64

## Exercises with statsmodels

- **For all problems: y is the interest rate of the loan**
- I recommend the _statsmodels formula_ method on the website

Psuedocode for using statsmodels to run a regression:
```python
model = sm_ols(<formula>, data=<dataframe>)
result=model.fit()

# to print regression output: result.summary()
# get predicted values (yhat): result.predict
# get regression residuals (uhat): result.resid
```

### Q1: Starter regressions

A. Regress y on the credit score (student demo): $y=\beta_0 + \beta_1*\text{Credit Score}$
- _I'll show 2 ways: the psuedo code and the one-liner_

B. Regress y on the **natural log** of the credit score: $y=\beta_0 + \beta_1*log(\text{Credit Score})$
- _I'll show two ways to do this_

C. Regress y on the **natural log** of the loan-to-value

D. Regress y on the natural log of the loan-to-value and the natural log of the credit score: $y=\beta_0 + \beta_1*log(\text{LTV}) + \beta_2*log(\text{Credit Score})$

In [50]:
from statsmodels.formula.api import ols as sm_ols
model=sm_ols(
    "Original_Interest_Rate ~ Borrower_Credit_Score_at_Origination",data=fannie_mae).fit().summary()
#result=model.fit()
#result.summary()
model



<class 'statsmodels.iolib.summary.Summary'>
"""
                              OLS Regression Results                              
==================================================================================
Dep. Variable:     Original_Interest_Rate   R-squared:                       0.126
Model:                                OLS   Adj. R-squared:                  0.126
Method:                     Least Squares   F-statistic:                 1.938e+04
Date:                    Wed, 22 Mar 2023   Prob (F-statistic):               0.00
Time:                            11:44:56   Log-Likelihood:            -2.1575e+05
No. Observations:                  134481   AIC:                         4.315e+05
Df Residuals:                      134479   BIC:                         4.315e+05
Df Model:                               1                                         
Covariance Type:                nonrobust                                         
========================================================================================================
                                           coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------------------
Intercept                               11.5819      0.046    253.270      0.000      11.492      11.671
Borrower_Credit_Score_at_Origination    -0.0086   6.14e-05   -139.198      0.000      -0.009      -0.008
==============================================================================
Omnibus:                     2660.479   Durbin-Watson:                   0.397
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             2660.737
Skew:                           0.321   Prob(JB):                         0.00
Kurtosis:                       2.750   Cond. No.                     1.04e+04
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.04e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

### Q2: Best practices: Look at the outputs every time

Let's talk about the outputs you see and should look at EVERY time you run a regression:
- Number of obs  134k
- R2 12%
- AR2
- Coef 
- Std error, t value, p value ("P>|t|")
- Std error options:
    - `.fit(cov_type="HC2")`
    - `.fit(cov_type="cluster", cov_kwds={"groups": df["industry"]})`

In [37]:
len(fannie_mae)

135038

In [49]:
#1.b
model2=sm_ols(
    "Original_Interest_Rate ~np.log( Borrower_Credit_Score_at_Origination)",data=fannie_mae).fit().summary()
sm_ols(
    "Original_Interest_Rate ~l_credscore",data=fannie_mae).fit().summary()

#1.c
sm_ols(
    "Original_Interest_Rate ~l_LTV",data=fannie_mae).fit().summary()
sm_ols(
    "Original_Interest_Rate ~np.log(Q('Original_LTV_(OLTV)'))",data=fannie_mae).fit().summary()
sm_ols(
    "Original_Interest_Rate ~l_LTV+l_credscore",data=fannie_mae).fit().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                              OLS Regression Results                              
==================================================================================
Dep. Variable:     Original_Interest_Rate   R-squared:                       0.126
Model:                                OLS   Adj. R-squared:                  0.126
Method:                     Least Squares   F-statistic:                     9656.
Date:                    Wed, 22 Mar 2023   Prob (F-statistic):               0.00
Time:                            11:44:51   Log-Likelihood:            -2.1578e+05
No. Observations:                  134481   AIC:                         4.316e+05
Df Residuals:                      134478   BIC:                         4.316e+05
Df Model:                               2                                         
Covariance Type:                nonrobust                                         
===============================================================================
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
Intercept      44.1324      0.302    145.949      0.000      43.540      44.725
l_LTV           0.1546      0.010     14.765      0.000       0.134       0.175
l_credscore    -5.9859      0.044   -134.888      0.000      -6.073      -5.899
==============================================================================
Omnibus:                     2793.369   Durbin-Watson:                   0.386
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             2743.990
Skew:                           0.321   Prob(JB):                         0.00
Kurtosis:                       2.720   Cond. No.                         735.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

### Q3: Regressions with transformations

We are talking about "linear regression. What that means is that the model is linear in the regressors: but it doesn’t mean that those regressors can't be some kind of non-linear transform of the original features $x_i$." The most common transformations are logging variables, interaction terms, and polynomial terms."

We already did log transformations above. 

An interaction term simply means one regressor is two variables multiplied:
- $y=\beta_0 + \beta_1 x_1 + \beta_2 x_1 x_2$
- $y=\beta_0 + \beta_1 x_1 + \beta_2 x_2 + \beta_3 x_1 x_2 $

Polynomial terms might look like:
- $y=\beta_0 + \beta_1 x_1 + \beta_2 x_1^2$

A. Regress y on the credit score and the credit score squared. 

B. Regress y on the natural log of the loan-to-value, the natural log of the credit score, and the interaction of LTV and credit score. 



In [56]:
sm_ols("Original_Interest_Rate ~l_LTV+np.power(l_credscore,2)",data=fannie_mae).fit().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                              OLS Regression Results                              
==================================================================================
Dep. Variable:     Original_Interest_Rate   R-squared:                       0.126
Model:                                OLS   Adj. R-squared:                  0.126
Method:                     Least Squares   F-statistic:                     9681.
Date:                    Wed, 22 Mar 2023   Prob (F-statistic):               0.00
Time:                            11:55:10   Log-Likelihood:            -2.1576e+05
No. Observations:                  134481   AIC:                         4.315e+05
Df Residuals:                      134478   BIC:                         4.316e+05
Df Model:                               2                                         
Covariance Type:                nonrobust                                         
============================================================================================
                               coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------
Intercept                   24.5068      0.160    153.541      0.000      24.194      24.820
l_LTV                        0.1532      0.010     14.632      0.000       0.133       0.174
np.power(l_credscore, 2)    -0.4562      0.003   -135.073      0.000      -0.463      -0.450
==============================================================================
Omnibus:                     2780.551   Durbin-Watson:                   0.386
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             2732.875
Skew:                           0.320   Prob(JB):                         0.00
Kurtosis:                       2.722   Cond. No.                     2.14e+03
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 2.14e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [53]:
sm_ols("Original_Interest_Rate ~l_LTV+np.square(l_credscore)",data=fannie_mae).fit().summary()
sm_ols("Original_Interest_Rate ~l_LTV+l_credscore+l_LTV*l_credscore",data=fannie_mae).fit().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                              OLS Regression Results                              
==================================================================================
Dep. Variable:     Original_Interest_Rate   R-squared:                       0.127
Model:                                OLS   Adj. R-squared:                  0.127
Method:                     Least Squares   F-statistic:                     6521.
Date:                    Wed, 22 Mar 2023   Prob (F-statistic):               0.00
Time:                            11:51:24   Log-Likelihood:            -2.1567e+05
No. Observations:                  134481   AIC:                         4.313e+05
Df Residuals:                      134477   BIC:                         4.314e+05
Df Model:                               3                                         
Covariance Type:                nonrobust                                         
=====================================================================================
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
Intercept           -16.8119      4.111     -4.090      0.000     -24.869      -8.755
l_LTV                14.6120      0.973     15.024      0.000      12.706      16.518
l_credscore           3.2155      0.621      5.182      0.000       1.999       4.432
l_LTV:l_credscore    -2.1830      0.147    -14.866      0.000      -2.471      -1.895
==============================================================================
Omnibus:                     2756.628   Durbin-Watson:                   0.389
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             2719.875
Skew:                           0.321   Prob(JB):                         0.00
Kurtosis:                       2.727   Cond. No.                     3.77e+04
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 3.77e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

sm_ols("Original_Interest_Rate ~l_LTV+np.square(l_credscore)",data=fannie_mae).fit().summary()### Q4: Dummy and categorical variables

A. Regress y on the dummy variable for a great credit score.

B. Regress y on the categorical variable we created for credit bins.

C. (Advanced, optional, after class exercise): High dimensional fixed effects. This basically means "a categorical variable with LOTS of values". [See this discussion.](https://aeturrell.github.io/coding-for-economists/econmt-regression.html#high-dimensional-fixed-effects-aka-absorbing-regression)

In [55]:
sm_ols("Original_Interest_Rate ~great",data=fannie_mae).fit().summary()
sm_ols("Original_Interest_Rate ~C(creditbins)",data=fannie_mae).fit().summary() # force it knowing that it is categorical variable


<class 'statsmodels.iolib.summary.Summary'>
"""
                              OLS Regression Results                              
==================================================================================
Dep. Variable:     Original_Interest_Rate   R-squared:                       0.116
Model:                                OLS   Adj. R-squared:                  0.116
Method:                     Least Squares   F-statistic:                     4411.
Date:                    Wed, 22 Mar 2023   Prob (F-statistic):               0.00
Time:                            11:53:44   Log-Likelihood:            -2.1651e+05
No. Observations:                  134481   AIC:                         4.330e+05
Df Residuals:                      134476   BIC:                         4.331e+05
Df Model:                               4                                         
Covariance Type:                nonrobust                                         
================================================================================================
                                   coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------------
Intercept                        6.7172      0.048    140.160      0.000       6.623       6.811
C(creditbins)[T.Fair]           -0.6749      0.049    -13.784      0.000      -0.771      -0.579
C(creditbins)[T.Good]           -1.2020      0.048    -24.881      0.000      -1.297      -1.107
C(creditbins)[T.Very Good]      -1.6642      0.048    -34.552      0.000      -1.759      -1.570
C(creditbins)[T.Exceptional]    -2.2655      0.049    -46.351      0.000      -2.361      -2.170
==============================================================================
Omnibus:                     2410.734   Durbin-Watson:                   0.385
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             2345.942
Skew:                           0.294   Prob(JB):                         0.00
Kurtosis:                       2.729   Cond. No.                         37.9
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

### Q5: Summarize what you've learned so far


### Q6: Plot the regression

_If time is tight: I'll do it._

Plot 1:
- Plot a scatterplot: Plot as X the credit score variable. As Y, use our y.
- On top of that, lineplots:
    - Rerun Q1a's reg and plot the yhat values. 
    - Let's talk about this.
    - Rerun Q1b's reg and plot the yhat values.
    - Compare to the prior line.
    
Plot 2:
- Plot a scatterplot: Plot as X the credit score variable. As Y, use our y.
- On top of that, lineplots:
    - Rerun Q4b's reg and plot the yhat values, hued by credit bin
  
Plot 3:
- Plot a scatterplot: Plot as X the credit score variable. As Y, use our y.
- On top of that, lineplots:
    - Rerun Q4b's reg BUT WITH credit score as a variable and plot the yhat values, hued by credit bin  
    
_Note: statsmodels has some useful plotting functions. My favs are influence_plot (can be slow) and plot_partregress_grid._

## Regression with SKLEARN

I don't like running regressions in `sklearn` usually. The main reason to do so is if you're doing a typical ML task that sklearn excels in (meaning: "pipelines", which is a term you'll understand later in the course) or if you know you're going to be using other sklearn models anyways (in which case, you'll already be doing the set up for sklearn).

But I want to run at least one regression in SKLEARN for you so you can see how the mechanics are similar, and how they differ. We will cover sklearn more in future classes.

Psuedocode for a reg in sklearn is similar. The differences:
1. A little more work setting up the data
1. `.fit()` gets the data passed to it 
1. The `results` object is different than statsmodels'

```python

# 1. import the "class" of model form sklearn

from sklearn.linear_model import LinearRegression

# 2. arrange the data - more work than statsmodels

# Issue: sklearn doesn't work with missing values, so drop any obs with missing values
# replace vars_in_your_reg with a list of variables you want to use, including y
subset = df[vars_in_your_reg].dropna()

# explicitly set up the y variable and the X variables you want
y = subset['y'] # whatever the y variable is
X = subset[['X1','X2']] # list the X vars

# 3. set up the model ("instantiate the model")
# every class of models has "hyperparamaters" that control how you want the model to work
# below, fit_intercept=True is a "hyperparameter" for OLS models 
# hyperparameters are the things inside the parenthesis of the model class when you declare it

model = LinearRegression(fit_intercept=True)
result=model.fit(X,y) # in sklearn, you put X and Y inside fit!!!

# the result object is different in sklearn
# results.intercept_ (the constant in the model)
# results.coef_ (the other X vars)

```


## Q7: STUDENT DEMO - regressions **using sklearn**

A. Regress the interest rate on the natural log of the loan-to-value using the sklearn method.

B. Regress the interest rate on the natural log of the loan-to-value using the sklearn method.